In [1]:
import numpy as np
import sys

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Dropout, Input
from keras.optimizers import RMSprop, Adam

from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import tensorflow as tf

Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.2.1
2.0.6


## Load the corpus

In [3]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

## Make look up tables based on characters

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Unique chars: ", n_vocab)

Total Characters:  144343
Total Unique chars:  44


In [5]:
#raw_text = raw_text[1:]

## Make the sequences

This shows you an example of making sequences sampled from the overall text data. 

We are creating sequences that are 100 characters long

In [6]:
# create input and output pairs
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total sequences: ", n_patterns)

Total sequences:  144243


### Lets examine some of these sequences

In [7]:
print(dataX[0])
print(dataY[0])

[43, 19, 24, 17, 32, 36, 21, 34, 1, 25, 10, 1, 20, 31, 39, 30, 1, 36, 24, 21, 1, 34, 17, 18, 18, 25, 36, 9, 24, 31, 28, 21, 0, 0, 17, 28, 25, 19, 21, 1, 39, 17, 35, 1, 18, 21, 23, 25, 30, 30, 25, 30, 23, 1, 36, 31, 1, 23, 21, 36, 1, 38, 21, 34, 41, 1, 36, 25, 34, 21, 20, 1, 31, 22, 1, 35, 25, 36, 36, 25, 30, 23, 1, 18, 41, 1, 24, 21, 34, 1, 35, 25, 35, 36, 21, 34, 1, 31, 30, 1]
36


In [8]:
print("\"", ''.join([int_to_char[value] for value in dataX[0]]), "\"")
print(int_to_char[dataY[0]])

" ﻿chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on  "
t


In [9]:
print(dataX[1])
print(dataY[1])

[19, 24, 17, 32, 36, 21, 34, 1, 25, 10, 1, 20, 31, 39, 30, 1, 36, 24, 21, 1, 34, 17, 18, 18, 25, 36, 9, 24, 31, 28, 21, 0, 0, 17, 28, 25, 19, 21, 1, 39, 17, 35, 1, 18, 21, 23, 25, 30, 30, 25, 30, 23, 1, 36, 31, 1, 23, 21, 36, 1, 38, 21, 34, 41, 1, 36, 25, 34, 21, 20, 1, 31, 22, 1, 35, 25, 36, 36, 25, 30, 23, 1, 18, 41, 1, 24, 21, 34, 1, 35, 25, 35, 36, 21, 34, 1, 31, 30, 1, 36]
24


In [10]:
print("\"", ''.join([int_to_char[value] for value in dataX[1]]), "\"")
print(int_to_char[dataY[1]])

" chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on t "
h


In [11]:
print("\"", ''.join([int_to_char[value] for value in dataX[4]]), "\"")
print(int_to_char[dataY[4]])

" pter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
 "
b


### Reshaping the sequences to become timesteps into the LSTM

In [12]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
print(X.shape)

# normalize
X = X / float(n_vocab)

(144243, 100, 1)


In [13]:
print(X[0][:10])
print(dataY[0])

[[ 0.97727273]
 [ 0.43181818]
 [ 0.54545455]
 [ 0.38636364]
 [ 0.72727273]
 [ 0.81818182]
 [ 0.47727273]
 [ 0.77272727]
 [ 0.02272727]
 [ 0.56818182]]
36


In [14]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [15]:
print(X[0][:10])
print(y[0])

[[ 0.97727273]
 [ 0.43181818]
 [ 0.54545455]
 [ 0.38636364]
 [ 0.72727273]
 [ 0.81818182]
 [ 0.47727273]
 [ 0.77272727]
 [ 0.02272727]
 [ 0.56818182]]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.]


## Creating our model

We will use the return sequences = true to pass the sequence up to the 2nd LSTM

In [16]:
# define the input shape
inp = Input(shape=(X.shape[1], X.shape[2]))
print('our input shape is ',(X.shape[1], X.shape[2]) )

our input shape is  (100, 1)


In [33]:
x = LSTM(256, return_sequences = True)(inp) 
#x = Dropout(0.2)(x)
x = LSTM(256)(x)
#x = Dropout(0.2)(x)
output = Dense(y.shape[1], activation ='softmax')(x)

In [34]:
generative_model = Model(inputs = inp, outputs=output )

In [35]:
optimizer = RMSprop(lr=0.01)
generative_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [36]:
# define the checkpoint
filepath="checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}-gentext-CharRNN.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [37]:
generative_model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
144243/144243 [==============================] - 315s - loss: 2.7150   
Epoch 2/10
144243/144243 [==============================] - 314s - loss: 2.2817   
Epoch 3/10
144243/144243 [==============================] - 311s - loss: 2.0553   
Epoch 4/10
144243/144243 [==============================] - 312s - loss: 1.9099   
Epoch 5/10
108992/144243 [=====================>........] - ETA: 76s - loss: 1.8108

KeyboardInterrupt: 

In [42]:
generative_model.save('Text_gen_01-CharRNN_no_embedding')

In [43]:
generative_model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
144243/144243 [==============================] - 313s - loss: 1.7415   
Epoch 2/10
144243/144243 [==============================] - 313s - loss: 1.6662   
Epoch 3/10
144243/144243 [==============================] - 313s - loss: 1.6029   
Epoch 4/10
144243/144243 [==============================] - 313s - loss: 1.5466   
Epoch 5/10
144243/144243 [==============================] - 313s - loss: 1.4981   
Epoch 6/10
144243/144243 [==============================] - 314s - loss: 1.4513   
Epoch 7/10
144243/144243 [==============================] - 313s - loss: 1.4079   
Epoch 8/10
144243/144243 [==============================] - 313s - loss: 1.3701   
Epoch 9/10
144243/144243 [==============================] - 314s - loss: 1.3345   
Epoch 10/10
144243/144243 [==============================] - 313s - loss: 1.3005   


In [44]:
generative_model.save('Text_gen_01-CharRNN_no_embedding')

In [57]:
#generative_model = load_model('Text_gen_01_no_embedding')

In [45]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144343
Total Vocab:  44


In [46]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
seed = dataX[start]
print(pattern)
print("Seed pattern:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

[8, 1, 36, 24, 17, 36, 1, 29, 17, 27, 21, 35, 1, 36, 24, 21, 1, 39, 31, 34, 28, 20, 1, 23, 31, 1, 34, 31, 37, 30, 20, 2, 3, 4, 0, 0, 4, 35, 31, 29, 21, 18, 31, 20, 41, 1, 35, 17, 25, 20, 8, 4, 1, 17, 28, 25, 19, 21, 1, 39, 24, 25, 35, 32, 21, 34, 21, 20, 8, 1, 4, 36, 24, 17, 36, 1, 25, 36, 4, 35, 1, 20, 31, 30, 21, 1, 18, 41, 1, 21, 38, 21, 34, 41, 18, 31, 20, 41, 1, 29]
Seed pattern:
" , that makes the world go round!"'

'somebody said,' alice whispered, 'that it's done by everybody m "


In [47]:
generated_text = []

# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    pattern.append(index)
    generated_text.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")


Done.


In [48]:
print(pattern)
print("\"", ''.join([int_to_char[value] for value in seed]), "\"")
print("\"", ''.join([int_to_char[value] for value in generated_text]), "\"")

[25, 30, 21, 1, 17, 18, 31, 37, 36, 1, 36, 24, 21, 1, 39, 24, 25, 36, 21, 1, 23, 17, 20, 1, 17, 1, 18, 31, 30, 27, 1, 36, 34, 1, 36, 24, 21, 1, 39, 31, 34, 20, 35, 1, 4, 1, 0, 4, 25, 1, 20, 31, 30, 4, 36, 1, 27, 30, 31, 39, 1, 36, 24, 21, 1, 39, 24, 25, 36, 21, 1, 34, 37, 21, 21, 34, 1, 35, 24, 21, 1, 39, 17, 41, 1, 25, 36, 1, 25, 17, 20, 1, 36, 31, 1, 35, 21, 21, 1, 36]
" , that makes the world go round!"'

'somebody said,' alice whispered, 'that it's done by everybody mi "
" ine about the white gad a bonk tr the words ' 
'i don't know the white rueer she way it iad to see t "


In [49]:
generated_text = []

# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    pattern.append(index)
    generated_text.append(index)
    pattern = pattern[1:len(pattern)]
print(pattern)
print("\"", ''.join([int_to_char[value] for value in seed]), "\"")
print("\"", ''.join([int_to_char[value] for value in generated_text]), "\"")

[24, 21, 1, 39, 17, 41, 1, 25, 30, 36, 31, 1, 36, 24, 21, 1, 21, 17, 30, 19, 21, 13, 0, 1, 1, 39, 31, 37, 28, 20, 1, 41, 31, 37, 1, 36, 21, 28, 28, 1, 29, 21, 1, 41, 31, 37, 1, 23, 17, 20, 1, 36, 31, 1, 35, 21, 21, 1, 36, 24, 21, 1, 39, 17, 41, 1, 25, 30, 1, 36, 24, 21, 1, 39, 17, 41, 1, 25, 30, 1, 36, 24, 21, 1, 39, 17, 41, 1, 25, 30, 1, 36, 24, 21, 1, 39, 17, 41, 1, 25]
" , that makes the world go round!"'

'somebody said,' alice whispered, 'that it's done by everybody mi "
" he way into the eance?
  would you tell me you gad to see the way in the way in the way in the way i "


In [50]:
generated_text = []

# generate characters
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    pattern.append(index)
    generated_text.append(index)
    pattern = pattern[1:len(pattern)]
print(pattern)
print("\"", ''.join([int_to_char[value] for value in seed]), "\"")
print("\"", ''.join([int_to_char[value] for value in generated_text]), "\"")

[1, 18, 21, 1, 17, 1, 32, 21, 36, 19, 34, 1, 24, 34, 31, 39, 1, 36, 24, 21, 1, 39, 24, 25, 36, 21, 1, 31, 17, 20, 21, 1, 31, 22, 1, 36, 24, 21, 1, 35, 31, 31, 28, 8, 1, 17, 30, 20, 1, 36, 24, 21, 1, 29, 31, 19, 35, 36, 21, 34, 35, 1, 25, 1, 24, 17, 38, 21, 1, 20, 31, 30, 21, 1, 36, 31, 1, 36, 24, 21, 1, 39, 31, 34, 20, 35, 1, 24, 31, 36, 1, 36, 31, 1, 36, 24, 21, 1, 39, 31]
" , that makes the world go round!"'

'somebody said,' alice whispered, 'that it's done by everybody mi "
" n the way in the way of starpelte that she had to sto the white rabbit heard the white rabbit, who was she white rabbit reterely at the white rabbit, whth a soor lany and frownen to the words the whole party things at the way of say that she was not and seneating the white rabbit, who was she white rabbit reterpadly, 'i should think you con't know the white giddres,' she said to herself, 'it would be a petcr hrow the white oade of the sool, and the mocsters i have done to the words hot to the wo "
